In [6]:
import json
from datasets import load_dataset
from trl import SFTConfig, SFTTrainer
from peft import LoraConfig

In [2]:
transcripts = {}
with open("./Labels/interview_transcripts_by_turkers.csv") as file:
        for curr in file:
                currsplit = curr.split(",")
                concat_string = ",".join(currsplit[1:])
                concat_string = concat_string.split("|")
                transcripts[(currsplit[0])] = concat_string
with open("processed_transcript.json", 'w') as fp:
    transcripts_data= []
    for curr_transcript_key in transcripts.keys():
        curr_transcript = []
        for curr_line in transcripts[curr_transcript_key]:
            curr_line = curr_line.split(": ")
            curr_transcript.append({'role': "assistant" if curr_line[0] == "Interviewer" else "user", 'content': curr_line[-1]})
        transcripts_data.append({"messages":curr_transcript})
    json.dump(transcripts_data, fp)

In [3]:
with open("processed_transcript.json", 'r') as jsonfile:
    data = json.load(jsonfile)

In [4]:
ds = load_dataset('json', data_files='processed_transcript.json')

Generating train split: 0 examples [00:00, ? examples/s]